<a href="https://colab.research.google.com/github/anjuli0802/SongRecommendationChatBot/blob/main/SongRecommChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anvil-uplink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.2 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import anvil.server

anvil.server.connect("server_Q3FDU7VT3NHJ56ZKSMT6HYCV-25BVOMNN7IAOPR3M")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [3]:
# pip install tensorflow keras pickle nltk
from google.colab import drive
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open('/content/drive/MyDrive/intents.json').read()
intents = json.loads(intents_file)

In [5]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
import nltk
nltk.download('punkt')
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['hi'], 'greeting'), (['hello'], 'greeting'), (['whats', 'up'], 'greeting'), (['sup'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['whats', 'good'], 'greeting'), (['hey'], 'greeting'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['im', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['ty'], 'thanks'), (['I', 'owe', 'you', 'one'], 'thanks'), (['whats', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name'], 'name'), (['whats', 'should', 'I', 'call', 'you'], 'name'), (['how', 'should', 'I', 'address', 'you'], 'name'), (['Yes', 'it', 'does'], 'sky_net_yes'), (['Yeah'], 'sky_net_yes'), (['Haha', 'yep'], 'sky_net_yes'), (['yes'], 'sky_net_yes'), (['Indeed'], 'sky_net_yes'), (['Yup'], 'sky_net_yes'), (['Just', 'like', 'the', 'terminat

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
import nltk
nltk.download('wordnet')
# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)
# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words

    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training, dtype = object)
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

[nltk_data] Downloading package wordnet to /root/nltk_data...


Training data is created


In [8]:
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#Training and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")


Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0397 - loss: 3.1609
Epoch 2/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1178 - loss: 3.1310  
Epoch 3/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1579 - loss: 2.8829
Epoch 4/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1740 - loss: 2.9135     
Epoch 5/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2632 - loss: 2.8212     
Epoch 6/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2516 - loss: 2.5665 
Epoch 7/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2351 - loss: 2.6688     
Epoch 8/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3480 - loss: 2.3021
Epoch 9/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2847 - loss: 2.4255 
Epoch 10/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3665 - loss: 2.1325
Epoch 11/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3958 - loss: 2.0898 
Epoch 12/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - a

model is created


In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('/content/drive/MyDrive/intents.json').read())
words = pickle.load(open('/content/drive/MyDrive/Major Project 2/words.pkl','rb'))
classes = pickle.load(open('/content/drive/MyDrive/Major Project 2/classes.pkl','rb'))
def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))
def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


In [11]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('/content/drive/MyDrive/intents.json').read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype = object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=172, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


[(['hi'], 'greeting'), (['hello'], 'greeting'), (['whats', 'up'], 'greeting'), (['sup'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['whats', 'good'], 'greeting'), (['hey'], 'greeting'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['im', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['ty'], 'thanks'), (['I', 'owe', 'you', 'one'], 'thanks'), (['whats', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name'], 'name'), (['whats', 'should', 'I', 'call', 'you'], 'name'), (['how', 'should', 'I', 'address', 'you'], 'name'), (['Yes', 'it', 'does'], 'sky_net_yes'), (['Yeah'], 'sky_net_yes'), (['Haha', 'yep'], 'sky_net_yes'), (['yes'], 'sky_net_yes'), (['Indeed'], 'sky_net_yes'), (['Yup'], 'sky_net_yes'), (['Just', 'like', 'the', 'terminat

model created


In [12]:
import json

In [13]:
msg = list()
text = str()

In [14]:
@anvil.server.callable
def responsed(msg1):
    msg.append(msg1)
    ints = predict_class(msg1)
    res = getResponse(ints, intents)
    return res

In [15]:
from textblob import TextBlob
import requests

@anvil.server.callable
def song_emotion():
    # Combine the last five messages into one string
    len1 = len(msg)
    text_to_analyze = " ".join(msg[len1-5:len1])  # Get the last five messages

    # Analyze sentiment using TextBlob
    blob = TextBlob(text_to_analyze)
    sentiment = blob.sentiment.polarity  # Get the polarity score

    # Determine emotion based on polarity
    if sentiment > 0:
        emotion = "happy"
        url = f"http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag=happy&api_key=21f566bd048af3f28e57098250cc00ac&format=json&limit=10"
    elif sentiment < 0:
        emotion = "sad"
        url = f"http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag=sad&api_key=21f566bd048af3f28e57098250cc00ac&format=json&limit=10"
    else:
        emotion = "neutral"
        url = f"http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag=neutral&api_key=21f566bd048af3f28e57098250cc00ac&format=json&limit=10"

    dic1 = dict()
    dic1['emotion'] = emotion

    # Fetch top tracks based on the detected emotion
    # url = f"http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag=emotion&api_key=21f566bd048af3f28e57098250cc00ac&format=json&limit=10"
    response = requests.get(url)
    payload = response.json()

    # Add top tracks to the dictionary
    for i in range(min(10, len(payload['tracks']['track']))):  # Ensure we don't exceed available tracks
        r = payload['tracks']['track'][i]
        dic1[r['name']] = r['url']

    return dic1

In [16]:
import requests

# url=f"http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag=happy&api_key={api_key}&format=json&limit=5"
url=f"http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag=emotion&api_key=21f566bd048af3f28e57098250cc00ac&format=json&limit=10"
# url=f"http://www.last.fm/?method=tag.gettoptracks&tag=happy&api_key=21f566bd048af3f28e57098250cc00ac&format=json&limit=10"

response = requests.get(url)
payload = response.json()
# for i in range(4):
r=payload['tracks']['track'][0]
# print(r['url'])
print(payload)

{'tracks': {'track': [{'name': 'Good Luck, Babe!', 'duration': '0', 'mbid': '', 'url': 'https://www.last.fm/music/Chappell+Roan/_/Good+Luck,+Babe%21', 'streamable': {'#text': '0', 'fulltrack': '0'}, 'artist': {'name': 'Chappell Roan', 'mbid': '', 'url': 'https://www.last.fm/music/Chappell+Roan'}, 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'small'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'medium'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'large'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'extralarge'}], '@attr': {'rank': '1'}}, {'name': 'Photograph', 'duration': '258', 'mbid': '7dcbc1be-4257-4d64-b915-045dfb23ae88', 'url': 'https://www.last.fm/music/Ed+Sheeran/_/Photograph', 'streamable': {'#text': '0', 'fulltrack': '0'}, 'artist': {'name'

In [ ]:
print("Chatbot : Hey there, Wassup ?")
# responded function takes text of user and returns chatbot output
for i in range(3):
    m = input("User : ")
    res = responsed(m)
    print("Chatbot : "+res)
ans = song_emotion()
print("Emotion : "+ans['emotion'])

Chatbot : Hey there, Wassup ?
User : hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Chatbot : Hello human
User : how are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Chatbot : I'm always great. How are you?
User : I'm doing great
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Chatbot : That sounds nice
Emotion : sad


In [ ]:
# song_emotion function would return dictionary consisting of emotion and recommended songs
ans = song_emotion()
print("Emotion : "+ans['emotion'])
ans.pop('emotion')
lst = list(ans.keys())
print("Song Recommendations : ")
for i in range(10):
    print("Song_name : "+lst[i])
    print("Song_URL : "+ans[lst[i]])


Emotion : sad
Song Recommendations : 
Song_name : Creep
Song_URL : https://www.last.fm/music/Radiohead/_/Creep
Song_name : No Surprises
Song_URL : https://www.last.fm/music/Radiohead/_/No+Surprises
Song_name : The Scientist
Song_URL : https://www.last.fm/music/Coldplay/_/The+Scientist
Song_name : Fix You
Song_URL : https://www.last.fm/music/Coldplay/_/Fix+You
Song_name : Someone Like You
Song_URL : https://www.last.fm/music/Adele/_/Someone+Like+You
Song_name : Sparks
Song_URL : https://www.last.fm/music/Coldplay/_/Sparks
Song_name : Fake Plastic Trees
Song_URL : https://www.last.fm/music/Radiohead/_/Fake+Plastic+Trees
Song_name : Exit Music (for a Film)
Song_URL : https://www.last.fm/music/Radiohead/_/Exit+Music+(for+a+Film)
Song_name : How to Save a Life
Song_URL : https://www.last.fm/music/The+Fray/_/How+to+Save+a+Life
Song_name : I MISS YOU
Song_URL : https://www.last.fm/music/blink-182/_/I+MISS+YOU


In [17]:
!pip install Flask
!pip install pyngrok

In [18]:
!pip install flask-ngrok

In [ ]:
!pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.5 MB/s eta 0:00:00


In [ ]:
#new 4
from flask import Flask, request, jsonify, render_template_string
import random
import requests
from flask_ngrok import run_with_ngrok
import json
from google.colab import files
from pyngrok import ngrok
from textblob import TextBlob

# Upload the intents.json file
uploaded = files.upload()

# Load intents from the uploaded JSON file
with open('intents.json', 'r') as file:
    intents = json.load(file)

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

# Initialize message count and messages list
message_count = 0
messages = []

# Function to get a response based on user input
def get_response(user_message):
    global message_count
    message_count += 1  # Increment the message count
    messages.append(user_message)  # Store the message

    # Check if the message count has reached 5
    if message_count >= 5:
        return song_emotion()

    for intent in intents['intents']:
        for pattern in intent['patterns']:
            if user_message.lower() in pattern.lower():
                return random.choice(intent['responses'])

    return "I'm not sure how to respond to that. Can you ask something else?"

# Function to analyze sentiment and recommend songs
def song_emotion():
    # Combine the last five messages into one string
    text_to_analyze = " ".join(messages[-5:])  # Get the last five messages

    # Analyze sentiment using TextBlob
    blob = TextBlob(text_to_analyze)
    sentiment = blob.sentiment.polarity  # Get the polarity score

    # Determine emotion based on polarity
    if sentiment > 0:
        emotion = "happy"
    elif sentiment < 0:
        emotion = "sad"
    else:
        emotion = "neutral"

    # Fetch top tracks based on the detected emotion
    url = f"http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag={emotion}&api_key=21f566bd048af3f28e57098250cc00ac&format=json&limit=10"
    response = requests.get(url)

    # Check if the response is successful
    if response.status_code != 200:
        return "Sorry, I couldn't fetch song recommendations at the moment."

    payload = response.json()

    # Prepare the response string with clickable links
    response_str = f"Based on your last five messages, I detected the emotion: {emotion}. Here are some song recommendations:<br>"
    for i in range(min(10, len(payload['tracks']['track']))):  # Ensure we don't exceed available tracks
        r = payload['tracks']['track'][i]
        response_str += f'<a href="{r["url"]}" target="_blank">{r["name"]}</a><br>'

    return response_str

@app.route('/')
def home():
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Chatbot</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                margin: 0;
                padding: 0;
            }
            .container {
                width: 80%;
                max-width: 600px;
                margin: 50px auto;
                background-color: white;
                border-radius: 8px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                padding: 20px;
            }
            h1 {
                text-align: center;
                color: #333;
            }
            .chatbox {
                display : flex;
                flex-direction: column;
            }
            .messages {
                height: 300px;
                overflow-y: auto;
                padding: 10px;
                margin-bottom: 10px;
                border-radius: 4px;
                background-color: #f9f9f9;
            }
            .user-message {
                text-align: right;
                color: #007bff;
                margin: 5px 0;
                background-color: #e1f5fe;
                padding: 10px;
                border-radius: 10px;
                display: inline-block;
                max-width:  70%;
                margin-left: auto;
            }
            .chatbot-message {
                text-align: left;
                color: #28a745;
                margin: 5px 0;
                background-color: #d1e7dd;
                padding: 10px;
                border-radius: 10px;
                display: inline-block;
                max-width: 70%;
            }
            input[type="text"] {
                padding: 10px;
                border: 1px solid #ccc;
                border-radius: 4px;
                margin-bottom: 10px;
                width: calc(100% - 22px);
            }
            button {
                padding: 10px;
                background-color: #007bff;
                color: white;
                border: none;
                border-radius: 4px;
                cursor: pointer;
            }
            button:hover {
                background-color: #0056b3;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Chatbot</h1>
            <div class="chatbox">
                <div class="messages" id="messages"></div>
                <input type="text" id="userInput" placeholder="Type a message..." />
                <button onclick="sendMessage()">Send</button>
            </div>
        </div>
        <script>
            function sendMessage() {
                var userInput = document.getElementById("userInput").value;
                if (userInput.trim() === "") return;

                // Display user message
                var messagesDiv = document.getElementById("messages");
                var userMessageDiv = document.createElement("div");
                userMessageDiv.className = "user-message";
                userMessageDiv.innerText = userInput;
                messagesDiv.appendChild(userMessageDiv);

                // Clear input
                document.getElementById("userInput").value = "";

                // Send message to server
                fetch("/get_response", {
                    method: "POST",
                    headers: {
                        "Content-Type": "application/json"
                    },
                    body: JSON.stringify({ message: userInput })
                })
                .then(response => response.json())
                .then(data => {
                    // Display chatbot response
                    var chatbotMessageDiv = document.createElement("div");
                    chatbotMessageDiv.className = "chatbot-message";
                    chatbotMessageDiv.innerHTML = data.response; // Use innerHTML to render links
                    messagesDiv.appendChild(chatbotMessageDiv);
                    messagesDiv.scrollTop = messagesDiv.scrollHeight; // Scroll to bottom
                });
            }
        </script>
    </body>
    </html>
    ''')

@app.route('/get_response', methods=['POST'])
def get_response_route():
    user_message = request.json['message']
    response = get_response(user_message)
    return jsonify({'response': response})

if __name__ == '__main__':
    # Set the ngrok authtoken if you have one
    ngrok.set_auth_token("2pDjXysk2mQfrbp6l9Rb9vu41tD_5LJTYwfmWbMWmm1h8ecsH")
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\"")
    app.run()

Saving intents.json to intents (7).json
 * ngrok tunnel "NgrokTunnel: "https://45ba-34-86-250-16.ngrok-free.app" -> "http://localhost:5000""
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://45ba-34-86-250-16.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [09/Dec/2024 19:26:45] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2024 19:26:45] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2024 19:26:50] "POST /get_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2024 19:28:03] "POST /get_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2024 19:29:06] "POST /get_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2024 19:30:48] "POST /get_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2024 19:31:35] "POST /get_response HTTP/1.1" 200 -
